In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Random processes creation

In [19]:
# (burst, arrival)
def create_processes(n):
    return np.random.randint(0, 10, size=(n,2))
processes = create_processes(100)

In [20]:
process_1 = np.array([
    [24,1],
    [5,2],
    [3,3],
])

### FCFS

In [21]:
def fcfs(processes):
    n = len(processes)
    rows = []
    total_time = 0
    processes = processes[processes[:,1].argsort()]
    process = 0
    print(processes)
    for burst, arrival in processes:
        if process == 0:
            wait_time = 0
            total_time += burst + arrival
            process += 1
        else:
            wait_time = total_time - arrival
            total_time += burst    
        turnaround_time = wait_time + burst
        print('Process {}: wait time = {}, turnaround time = {}, total_time = {}'.format(process, wait_time, turnaround_time, total_time))
        rows.append([burst, arrival,wait_time, turnaround_time])
    rows = np.array(rows)
    return {
        'Gantt Chart': rows,
        'Average Wait Time': rows[:,2].mean(),
        'Average Turnaround Time': rows[:,3].mean(),
    }


In [22]:
fcfs(process_1)

[[24  1]
 [ 5  2]
 [ 3  3]]
Process 1: wait time = 0, turnaround time = 24, total_time = 25
Process 1: wait time = 23, turnaround time = 28, total_time = 30
Process 1: wait time = 27, turnaround time = 30, total_time = 33


{'Gantt Chart': array([[24,  1,  0, 24],
        [ 5,  2, 23, 28],
        [ 3,  3, 27, 30]]),
 'Average Wait Time': 16.666666666666668,
 'Average Turnaround Time': 27.333333333333332}

### SJF

In [23]:
process_1 = np.array([
    [5,2],
    [4,3],
    [24,1],
])


In [24]:
def sjf(processes):
    def order_inside(processes, time):
        new_processes = []
        last_processes = []
        for process in processes:
            process = process.tolist()
            if process[1] <= time:
                new_processes.append(process)
            else:
                last_processes.append(process)
        new_processes.sort()
        return np.array(new_processes + last_processes)
    n = len(processes)
    rows = []
    total_time = 0
    processes = processes[processes[:,1].argsort()]
    process = 0
    for i in range(len(processes)):
        if process == 0:
            (burst, arrival), processes = processes[0], processes[1:]
            wait_time = 0
            total_time += burst + arrival
            process += 1
        else:
            processes = order_inside(processes, total_time)
            (burst, arrival), processes = processes[0], processes[1:]
            wait_time = total_time - arrival
            total_time += burst    
        turnaround_time = wait_time + burst
        print('Process {}: wait time = {}, turnaround time = {}, total_time = {}'.format(process, wait_time, turnaround_time, total_time))
        rows.append([burst, arrival,wait_time, turnaround_time])
    rows = np.array(rows)
    return {
        'Gantt Chart': rows,
        'Average Wait Time': rows[:,2].mean(),
        'Average Turnaround Time': rows[:,3].mean(),
    }


In [25]:
sjf(process_1)

Process 1: wait time = 0, turnaround time = 24, total_time = 25
Process 1: wait time = 22, turnaround time = 26, total_time = 29
Process 1: wait time = 27, turnaround time = 32, total_time = 34


{'Gantt Chart': array([[24,  1,  0, 24],
        [ 4,  3, 22, 26],
        [ 5,  2, 27, 32]]),
 'Average Wait Time': 16.333333333333332,
 'Average Turnaround Time': 27.333333333333332}

In [26]:
class process():
    def __init__(self, id, burst, arrival):
        self.id = id
        self.burst = burst
        self.arrival = arrival
        self.wait_time = 0
        self.turnaround_time = 0
        self.left = burst

    def __repr__(self):
        return 'id: {}, burst:{}, arrival: {}, wait time: {}, turnaround :{}'.format(self.id, self.burst, self.arrival, self.wait_time, self.turnaround_time)

    def to_numpy(self):
        return np.array([self.id, self.burst, self.arrival, self.wait_time, self.turnaround_time, self.left])

In [45]:
def srtf(processes):
    #Helper functions
    def process_creation(processes):
        i = 0
        all_processes = []
        for burst, arrival in processes:
            all_processes.append(process(i, burst, arrival))
            i += 1
        return all_processes
    def add_all(processes, total_time, except_id=None):
        if except_id is None:
            return
        for process in processes:
            if process[0] != except_id and process[2] < total_time:
                process[3] += 1
        return processes

    def subs_actual(processes, id):
        for process in processes:
            if process[0] == id:
                process[-1] -= 1
        return processes

    def depurate(processes):
        actual = []
        final = []
        for process in processes:
            if process[-1] > 0:
                actual.append(process)
            else:
                process[-2] = process[1] + process[3]
                final.append(process)
        return np.array(actual), np.array(final)
    processes = process_creation(processes)
    processes = np.array([ex.to_numpy() for ex in processes])
    processes = processes[processes[:,2].argsort()]
    total_time = 0
    actual_id = processes[0][0]
    final = []
    while True:
        processes = subs_actual(processes, actual_id)
        processes = add_all(processes, total_time, except_id=actual_id)
        processes, new_final = depurate(processes)
        if len(new_final) > 0:
            final.extend(new_final)
        if len(processes) == 0:
            total_time+=1
            break
        processes = processes[np.lexsort((processes[:,2],processes[:,1], processes[:,-1]))]
        if processes[0][2] <= total_time:
            actual_id = processes[0][0]
        total_time+=1
    final = np.array(final)
    print(final)
    return {
        'Gantt Chart': final[:,:-1],
        'Average Wait Time': final[:,3].mean(),
        'Average Turnaround Time': final[:,4].mean(),
    }
    

In [46]:
srtf(process_1)

[[ 1  4  3  0  4  0]
 [ 0  5  2  5 10  0]
 [ 2 24  1  9 33  0]]


C:\Users\gerda\AppData\Local\Temp/ipykernel_12212/1021130965.py:44: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if new_final != []:
C:\Users\gerda\AppData\Local\Temp/ipykernel_12212/1021130965.py:44: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if new_final != []:


{'Gantt Chart': array([[ 1,  4,  3,  0,  4],
        [ 0,  5,  2,  5, 10],
        [ 2, 24,  1,  9, 33]]),
 'Average Wait Time': 4.666666666666667,
 'Average Turnaround Time': 15.666666666666666}